In [5]:
import fasta_subseq_2 as fa
import seq_plotmethods as sp
import itertools as it
import tables as tb
import numpy as np
import subprocess as subp


Initial Data Processing
-
- Start w/ gzipped .fastq files
- Generate scripts to map reads using bowtie2 
- Generate scripts to build h5 tables from mapped reads
    - **RESIST THE TEMPTATION TO RE-CODE ANY OF THESE THINGS**
- Generate scripts to run macs2 (GrizzlyPeaks?) for peak calling

In [3]:
# Working directory; change if necessary
current_dir = "[FILL_ME_IN]"
genomes_dir = current_dir + "genomes/"
reads_dir = current_dir + "reads/"
data_out_dir = current_dir + "data/"
figs_dir = current_dir + "figs/"
scripts_dir = current_dir + "scripts/"
mapping_dir = data_out_dir + "mapping/"
peaks_dir = data_out_dir + "peaks/"
h5_dir = data_out_dir + "hf5/"

# Dict of files to analyze
to_analyze = {'gtA':'Ant_Gt_ChIP.fastq.gz',
              'inA':'Ant_Input.fastq.gz',
              'gtC':'Combo_Gt_ChIP.fastq.gz',
              'inC':'Combo_Input.fastq.gz',
              'gtP':'Post_Gt_ChIP.fastq.gz',
              'inP':'Post_Input.fastq.gz',
              'gtW1':'Whole1_Gt_ChIP.fastq.gz',
              'inW1':'Whole1_Input.fastq.gz',
              'gtW2':'Whole2_Gt_ChIP.fastq.gz',
              'inW2':'Whole2_Input.fastq.gz',
              }

# sample / input pairs
# note that I'm just using concatenated inputs for all dmel samples since few reads
smp_in = {'gtA':'inA',
          'gtC':'inC',
          'gtP':'inP',
          'gtW1':'inW1',
          'gtW2':'inW2',
          }

In [6]:
# Build shell script to run bowtie2
bt2_genome_index = current_dir + "genomes/dmel+dpse"

bowtie2_cmd = "bowtie2 --end-to-end \
--un-gz %s \
--no-unal \
--mm \
--qc-filter \
-x %s -U %s -S %s 1> %s 2> %s"

run_bowtie_sh = open(scripts_dir + "run_bowtie.sh","w+")
header = """#!/bin/bash

#####################################
#
# bash script to run bowtie2 on .fastq files in ../reads dir
#
"""

print >> run_bowtie_sh, header
for (name,readfile) in to_analyze.items():
    print >> run_bowtie_sh, "echo \"Starting %s...\"" % name
    print >> run_bowtie_sh, bowtie2_cmd % (mapping_dir+name+"_unaligned.fastq.gz",
                                           bt2_genome_index,
                                           reads_dir+readfile,
                                           mapping_dir+name+"_aln.sam",
                                           mapping_dir+name+"_stdout.txt",
                                           mapping_dir+name+"_stderr.txt")
    print >> run_bowtie_sh, "echo \"Done!\""
run_bowtie_sh.close()
subp.call(['chmod','a+x',scripts_dir + 'run_bowtie.sh'])

0

In [7]:
# grep out dmel & dpse samfiles
grep_dmdp_sh = open(scripts_dir + "grep_dmdp.sh","w+")
header = """#!/bin/bash


#####################################
#
# bash script to grep dmel & dpse called reads from bowtie2 sam file 
#
"""
print >> grep_dmdp_sh, header
for name in to_analyze.keys():
    samfile = mapping_dir+name+"_aln.sam"
    print >> grep_dmdp_sh, "echo '%s...'" % (name,)
    print >> grep_dmdp_sh, "grep \"\tdpse_\" %s | perl -pe 's/dpse_//g' | grep -v 'mitochondrion_genome' | grep -v 'Unknown' | gzip > %s" % (samfile,mapping_dir+name+"_dpse_aln.sam.gz")
    print >> grep_dmdp_sh, "grep \"\tdmel_\" %s | perl -pe 's/dmel_//g' | grep -v 'mitochondrion_genome' | gzip > %s" % (samfile,mapping_dir+name+"_dmel_aln.sam.gz")
    print >> grep_dmdp_sh, "echo 'done!'"
grep_dmdp_sh.close()
subp.call(['chmod','a+x',scripts_dir + 'grep_dmdp.sh'])

0

In [8]:
# run ChIP normalization scripts
chip_norm_sh = open(scripts_dir + "chip_norm.sh","w+")
header = """#!/bin/bash


#####################################
#
# run chip_tagcount_normalize.py to generate pytables .hf5's for all data 
#
"""
print >> chip_norm_sh, header
for name in to_analyze.keys():
    print >> chip_norm_sh, "echo 'normalizing %s...'" % (name,)
    print >> chip_norm_sh, "chip_tagcount_normalize.py -n %s -q 30 -e 300 -o -u -z %s %s 1> %s 2> %s" % (name+"_dmel",genomes_dir+"dmel.fa",mapping_dir+name+"_dmel_aln.sam.gz",mapping_dir+name+"_dmel_tagnorm.stdout.txt",mapping_dir+name+"_dmel_tagnorm.stderr.txt")
    print >> chip_norm_sh, "chip_tagcount_normalize.py -n %s -q 30 -e 300 -o -u -z %s %s 1> %s 2> %s" % (name+"_dpse",genomes_dir+"dpse.fa",mapping_dir+name+"_dpse_aln.sam.gz",mapping_dir+name+"_dpse_tagnorm.stdout.txt",mapping_dir+name+"_dpse_tagnorm.stderr.txt")
    print >> chip_norm_sh, "echo 'done!'"
chip_norm_sh.close()
subp.call(['chmod','a+x',scripts_dir + 'chip_norm.sh'])


0

In [9]:
# use macs2 to call peaks
chip_norm_sbatch = open(scripts_dir + "chip_norm_sbatch.sh","w+")
tacc_dir = "[FILL_ME_IN]"
tacc_scripts = tacc_dir + "scripts/"
tacc_h5 = tacc_dir + "hf5/"
tacc_mapping = tacc_dir + "mapping/"
tacc_genomes = tacc_dir + "genomes/"
dm_mfold = (5,10)
dp_mfold = (5,10)
print >> chip_norm_sbatch, "#!/bin/bash"
print >> chip_norm_sbatch, "# SLURM job submission script for chip_norm.sh"

header = """#!/bin/bash

#####################################
#
# run chip_tagcount_normalize.py to generate pytables .hf5's for all data 
#
#####################################
# Job parameters:
#SBATCH -J %s           
#SBATCH -o %s.out%%j       
#SBATCH -e %s.err%%j 
#SBATCH -p normal
#SBATCH -n 1
#SBATCH -t 01:30:00
#SBATCH --mail-user=user@nowhere
#SBATCH --mail-type=begin  
#SBATCH --mail-type=end    
"""

for name in to_analyze.keys():
    """
    call_peaks_msh = open(scripts_dir + "chip_norm_%sm.sh" % name,"w+")
    print >> call_peaks_msh, header % (name+"m",name+"m",name+"m")
    print >> call_peaks_msh, "cd %s" % (tacc_h5,)
    print >> call_peaks_msh, "macs2 callpeak -t %s -c %s -n %s -f SAM -g dm -m %d %d" % (tacc_h5+name+"_dmel_aln.sam.gz",tacc_h5+"inAll_dmel_aln.sam.gz",name+"_dmel",dm_mfold[0],dm_mfold[1])
    call_peaks_msh.close()
    print >> call_peaks_sbatch, "sbatch call_peaks_%sm.sh" % (name,)
    """
    chip_norm_psh = open(scripts_dir + "chip_norm_%sp.sh" % name,"w+")
    print >> chip_norm_psh, header % (name+"p",tacc_h5+name+"p",tacc_h5+name+"p")
    print >> chip_norm_psh, "cd %s" % (tacc_h5,)
    print >> chip_norm_psh, "chip_tagcount_normalize.py -n %s -q 30 -e 300 -o -u -z %s %s" % (tacc_h5+name+"_dpse",tacc_genomes+"dpse.fa",tacc_mapping+name+"_dpse_aln.sam.gz")
    chip_norm_psh.close()
    print >> chip_norm_sbatch, "sbatch chip_norm_%sp.sh" % (name,)

chip_norm_sbatch.close()
subp.call(['chmod','a+x',scripts_dir + 'chip_norm_sbatch.sh'])

0

In [10]:
# use macs2 to call peaks
call_peaks_sh = open(scripts_dir + "call_peaks.sh","w+")
dm_mfold = (5,10)
dp_mfold = (5,10)
header = """#!/bin/bash


#####################################
#
# run chip_tagcount_normalize.py to generate pytables .hf5's for all data 
#
"""
print >> call_peaks_sh, header
print >> call_peaks_sh, "cd %s" % (mapping_dir,)
for name in smp_in.keys():
    print >> call_peaks_sh, "echo 'calling peaks for %s...'" % (name,)
    print >> call_peaks_sh, "macs2 -t %s -c %s -n %s -f SAM -g dm -m %d,%d 1> %s 2> %s" % (mapping_dir+name+"_dmel_aln.sam.gz",mapping_dir+"inAll_dmel_aln.sam.gz",name+"_dmel",dm_mfold[0],dm_mfold[1],peaks_dir+name+"_dmel_macs.stdout.txt",peaks_dir+name+"_dmel_macs.stderr.txt")
    print >> call_peaks_sh, "macs2 -t %s -c %s -n %s -f SAM -g 140000000 -m %d,%d 1> %s 2> %s" % (mapping_dir+name+"_dpse_aln.sam.gz",mapping_dir+smp_in[name]+"_dpse_aln.sam.gz",name+"_dpse",dp_mfold[0],dp_mfold[1],peaks_dir+name+"_dpse_macs.stdout.txt",peaks_dir+name+"_dpse_macs.stderr.txt")
    print >> call_peaks_sh, "echo 'done!'"
call_peaks_sh.close()
subp.call(['chmod','a+x',scripts_dir + 'call_peaks.sh'])

0

In [11]:
# use macs2 to call peaks
call_peaks_sbatch = open(scripts_dir + "call_peaks_sbatch.sh","w+")
tacc_dir = "[FILL_ME_IN]"
dm_mfold = (5,10)
dp_mfold = (5,10)
print >> call_peaks_sbatch, "#!/bin/bash"
print >> call_peaks_sbatch, "# SLURM job submission script for call_peaks.sh"

header = """#!/bin/bash

#####################################
#
# run chip_tagcount_normalize.py to generate pytables .hf5's for all data 
#
#####################################
# Job parameters:
#SBATCH -J %s           
#SBATCH -o %s.out%%j       
#SBATCH -e %s.err%%j 
#SBATCH -p normal
#SBATCH -n 1
#SBATCH -t 01:30:00
#SBATCH --mail-user=user@nowhere
#SBATCH --mail-type=begin  
#SBATCH --mail-type=end    
"""

for name in smp_in.keys():
    call_peaks_msh = open(scripts_dir + "call_peaks_%sm.sh" % name,"w+")
    print >> call_peaks_msh, header % (name+"m",name+"m",name+"m")
    print >> call_peaks_msh, "cd %s" % (tacc_dir+"peaks/",)
    print >> call_peaks_msh, "macs2 callpeak -t %s -c %s -n %s -f SAM -g dm -m %d %d" % (tacc_dir+'mapping/'+name+"_dmel_aln.sam.gz",tacc_dir+'mapping/'+"inAll_dmel_aln.sam.gz",name+"_dmel",dm_mfold[0],dm_mfold[1])
    call_peaks_msh.close()
    print >> call_peaks_sbatch, "sbatch call_peaks_%sm.sh" % (name,)
    call_peaks_psh = open(scripts_dir + "call_peaks_%sp.sh" % name,"w+")
    print >> call_peaks_psh, header % (name+"p",name+"p",name+"p")
    print >> call_peaks_psh, "cd %s" % (tacc_dir+"peaks/",)
    print >> call_peaks_psh, "macs2 callpeak -t %s -c %s -n %s -f SAM -g 140000000 -m %d %d" % (tacc_dir+'mapping/'+name+"_dpse_aln.sam.gz",tacc_dir+'mapping/'+smp_in[name]+"_dpse_aln.sam.gz",name+"_dpse",dp_mfold[0],dp_mfold[1])
    call_peaks_psh.close()
    print >> call_peaks_sbatch, "sbatch call_peaks_%sp.sh" % (name,)

call_peaks_sbatch.close()
subp.call(['chmod','a+x',scripts_dir + 'call_peaks_sbatch.sh'])

0

### Run scripts generated above in this order:
      1. `run_bowtie.sh`
      2. `grep_dmdp.sh`
      3. `chip_norm.sh`
      4. `call_peaks.sh` or `call_peaks_sbatch.sh`